In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(1, '/home/georgeos/Documents/GitHub/SyMBac/') # Not needed if you installed SyMBac using pip
sys.path.insert(1, '/home/georgeos/Documents/Github/SyMBac/') # Not needed if you installed SyMBac using pip


from SyMBac.drawing import raster_cell, OPL_to_FL
from SyMBac.PSF import PSF_generator
from SyMBac.renderer import convolve_rescale
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from PIL import Image
from skimage.util import img_as_uint
import os
import zarr
from numcodecs import Blosc
from joblib import Parallel, delayed
from glob import glob
from natsort import natsorted
from skimage.util import img_as_uint
from skimage.measure import label

In [3]:
def file_to_dict(filename):
    length, width, wavelength, name, NA, n =  filename.split("/")[1].split("_")
    length, width, wavelength, NA, n = float(length), float(width), float(wavelength), float(NA), float(n.split(".zarr")[0])

    data = {
        "length" : length,
        "width" : width,
        "wavelength" : wavelength,
        "NA": NA,
        "n" : n,
        "name" : name,
        "data" : zarr.open(filename)
           }
    
    return data

In [4]:
try:
    os.mkdir("synthetic_training_data")
except:
    pass

In [5]:
data_dirs = natsorted(glob("convolutions/*"))

In [6]:
def generate_mask_data_pair(data_dir):
    data = file_to_dict(data_dir)
    
    image = np.concatenate(data["data"], axis=1)

    cell_length = data["length"]
    cell_width = data["width"]
    resize_amount = 1
    pix_mic_conv = 0.065 #micron/pix
    raster_cell_length = cell_length/pix_mic_conv * resize_amount
    raster_cell_width = cell_width/pix_mic_conv * resize_amount
    densities = np.linspace(1,100, 10)

    cell = np.pad(raster_cell(raster_cell_length, raster_cell_width, separation=0), ((100,100),(200,200)))
    cell_mask = cell > 0
    
    mask = np.concatenate([cell_mask]*len(densities), axis=1)
    return img_as_uint(image), mask

In [13]:
aspect_ratios = np.unique(["_".join(x.split("/")[-1].split("_")[:2]) for x in data_dirs])
def training_data_generator(aspect_ratio, a):
    one_aspect_ratio = glob(f"convolutions/{aspect_ratio}_*")
    images, masks = [], []
    for data_dir in one_aspect_ratio:
        if np.random.rand() > 0.85:
            image, mask = generate_mask_data_pair(data_dir)
            images.append(image)
            masks.append(mask)
            
    image = np.vstack(images)
    mask = label(np.vstack(masks))
    image = Image.fromarray(image)
    mask = Image.fromarray(mask)
    image.save(f"synthetic_training_data/train_{str(a).zfill(4)}.png")
    mask.save(f"synthetic_training_data/train_{str(a).zfill(4)}_masks.png")
    


In [14]:
Parallel(n_jobs=-1)(delayed(training_data_generator)(aspect_ratio, a) for aspect_ratio, a in tqdm(zip(aspect_ratios, range(len(aspect_ratios))), total=len(aspect_ratios)))

  0%|          | 0/55 [00:00<?, ?it/s]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
a = 0

def training_data_generator(data_dir, a):
    image, mask = generate_mask_data_pair(data_dir)
    image = Image.fromarray(img_as_uint(image))
    mask = Image.fromarray(mask)
    image.save(f"synthetic_training_data/train_{str(a).zfill(4)}.png")
    mask.save(f"synthetic_training_data/train_{str(a).zfill(4)}_masks.png")
    
Parallel(n_jobs=-1)(delayed(training_data_generator)(data_dir, a) for data_dir, a in tqdm(zip(data_dirs, range(len(data_dirs))), total=len(data_dirs)))